In [1]:
import requests
import json
import pandas as pd
import logging
from collections import OrderedDict
import datetime



class ServingClient:
    def __init__(self):
        
        self.game_data_dict=OrderedDict()
        self.currentTime=pd.to_datetime(datetime.datetime.now())
        self.BASE_URL = "https://statsapi.web.nhl.com"
        self.SCHEDULE_URL = self.BASE_URL + "/api/v1/schedule"
        self.game_data_df=pd.DataFrame()

        
        
    def q1 (self,game_id : str = 2021020329 ,append=True):
        if game_id not in self.game_data_dict:
            response=requests.get(f'https://statsapi.web.nhl.com/api/v1/game/{game_id}/feed/live/')
            if response.reason=="OK":
                df = pd.json_normalize(response.json())
                if append:
                    self.game_data_dict[game_id]=df
                    self.game_data_df=pd.concat((self.game_data_df,df))
                return(response,df)
        else :
            print("this game data already exist in game data dict")
            
            
            
        
            
    def q2 (self,start_date=None, end_date=None):
        games=self.list_game(start_date, end_date)
        games_id=games.game_id.values
        
        games_id1=self.checkـexisting(games_id)
        
        df_lst=[]
        for game_id in games_id1:
            df_lst.append(self.q1(str(game_id),append=False)[1])
            
        res=pd.concat(df_lst)
        self.game_data_df=pd.concat((self.game_data_df,res))
        return(a.game_data_df.set_index("gamePk").loc[games_id].reset_index())


    
    def checkـexisting (self,games_id):
        return(set(games_id)-set(self.game_data_df.gamePk.values))
        
        

    def list_game(self,start_date=None, end_date=None):
        if start_date is None:
            start_date = str(datetime.date.today())
        if end_date is None:
            end_date = str(datetime.date.today())
        
        resp = requests.get(self.SCHEDULE_URL, params={"startDate": start_date, "endDate": end_date})
        data = resp.json()
        res=[self.game_summary(game) for date in data["dates"]for game in date["games"]]
        
        return (pd.DataFrame.from_dict(res))
            
    
    def game_summary(self,gs):
        return {"date": gs["gameDate"][:10],
                "game_id": gs["gamePk"],
                "home_team": gs["teams"]["home"]["team"]["name"],
                "home_score": gs["teams"]["home"]["score"],
                "away_team": gs["teams"]["away"]["team"]["name"],
                "away_score": gs["teams"]["away"]["score"],
                "season": gs["season"],
                "game_state": gs["status"]["detailedState"]}
        
            


a=ServingClient()
a.q1()
a.q2()




        
        

,gamePk,copyright,link,metaData.wait,metaData.timeStamp,gameData.game.pk,gameData.game.season,gameData.game.type,gameData.datetime.dateTime,gameData.datetime.endDateTime,gameData.status.abstractGameState,gameData.status.codedGameState,gameData.status.detailedState,gameData.status.statusCode,gameData.status.startTimeTBD,gameData.teams.away.id,gameData.teams.away.name,gameData.teams.away.link,gameData.teams.away.venue.id,gameData.teams.away.venue.name,gameData.teams.away.venue.link,gameData.teams.away.venue.city,gameData.teams.away.venue.timeZone.id,gameData.teams.away.venue.timeZone.offset,gameData.teams.away.venue.timeZone.tz,gameData.teams.away.abbreviation,gameData.teams.away.triCode,gameData.teams.away.teamName,gameData.teams.away.locationName,gameData.teams.away.firstYearOfPlay,gameData.teams.away.division.id,gameData.teams.away.division.name,gameData.teams.away.division.nameShort,gameData.teams.away.division.link,gameData.teams.away.division.abbreviation,gameData.teams.away.conference.id,gameData.teams.away.conference.name,gameData.teams.away.conference.link,gameData.teams.away.franchise.franchiseId,gameData.teams.away.franchise.teamName,...,liveData.boxscore.teams.home.players.ID8480039.stats.skaterStats.goals,liveData.boxscore.teams.home.players.ID8480039.stats.skaterStats.shots,liveData.boxscore.teams.home.players.ID8480039.stats.skaterStats.hits,liveData.boxscore.teams.home.players.ID8480039.stats.skaterStats.powerPlayGoals,liveData.boxscore.teams.home.players.ID8480039.stats.skaterStats.powerPlayAssists,liveData.boxscore.teams.home.players.ID8480039.stats.skaterStats.penaltyMinutes,liveData.boxscore.teams.home.players.ID8480039.stats.skaterStats.faceOffWins,liveData.boxscore.teams.home.players.ID8480039.stats.skaterStats.faceoffTaken,liveData.boxscore.teams.home.players.ID8480039.stats.skaterStats.takeaways,liveData.boxscore.teams.home.players.ID8480039.stats.skaterStats.giveaways,liveData.boxscore.teams.home.players.ID8480039.stats.skaterStats.shortHandedGoals,liveData.boxscore.teams.home.players.ID8480039.stats.skaterStats.shortHandedAssists,liveData.boxscore.teams.home.players.ID8480039.stats.skaterStats.blocked,liveData.boxscore.teams.home.players.ID8480039.stats.skaterStats.plusMinus,liveData.boxscore.teams.home.players.ID8480039.stats.skaterStats.evenTimeOnIce,liveData.boxscore.teams.home.players.ID8480039.stats.skaterStats.powerPlayTimeOnIce,liveData.boxscore.teams.home.players.ID8480039.stats.skaterStats.shortHandedTimeOnIce,liveData.boxscore.teams.home.goalies,liveData.boxscore.teams.home.skaters,liveData.boxscore.teams.home.onIce,liveData.boxscore.teams.home.onIcePlus,liveData.boxscore.teams.home.scratches,liveData.boxscore.teams.home.penaltyBox,liveData.boxscore.teams.home.coaches,liveData.boxscore.officials,liveData.decisions.winner.id,liveData.decisions.winner.fullName,liveData.decisions.winner.link,liveData.decisions.loser.id,liveData.decisions.loser.fullName,liveData.decisions.loser.link,liveData.decisions.firstStar.id,liveData.decisions.firstStar.fullName,liveData.decisions.firstStar.link,liveData.decisions.secondStar.id,liveData.decisions.secondStar.fullName,liveData.decisions.secondStar.link,liveData.decisions.thirdStar.id,liveData.decisions.thirdStar.fullName,liveData.decisions.thirdStar.link
0,2021020451,NHL and the NHL Shield are registered trademar...,/api/v1/game/2021020451/feed/live,10,20211207_000357,2021020451,20212022,R,2021-12-16T01:00:00Z,NaN,Preview,1,Scheduled,1,False,15,Washington Capitals,/api/v1/teams/15,5094.0,Capital One Arena,/api/v1/venues/5094,Washington,America/New_York,-5,EST,WSH,WSH,Capitals,Washington,1974,18,Metropolitan,Metro,/api/v1/divisions/18,M,6,Eastern,/api/v1/conferences/6,24,Capitals,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],[],[],[],[],[],[],[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021020452,NHL and the NHL Shield are registered trademar...,/api/v1/game/2021020452/feed/live,10,20211208_101459,2021020452,202